QUESTÃO 01

Importações Necessárias

In [1]:
from PIL import Image
from statistics import *
import random
from math import *
from typing import Final

qtdMaksKuwahara: Final = 4
qtdMaksTomitaTsuji: Final = 5
qtdMaksNagaoMatsuyama: Final = 9
qtdMaksSomboonkaew: Final = 12

Realce

In [2]:
def filtroRealce(Input:Image, tamJanela:int) -> Image:
    Output = Input.copy()
    tamInput = Input.size
    for x in range(tamJanela - 1, tamInput[0] - tamJanela + 1):
        for y in range(tamJanela - 1, tamInput[1] - tamJanela + 1):
            scan = []

            for i in range((-1) * floor(tamJanela / 2), ceil(tamJanela / 2)):
                for j in range((-1) * floor(tamJanela / 2), ceil(tamJanela / 2)):
                    scan.append(Input.getpixel((x + i, y + j)))

            Output.putpixel((x, y), median(scan))

    return Output

Suavização

In [3]:
def filtroMedia(Input:Image, tamJanela:int) -> Image:
    Output = Input.copy()
    tamInput = Input.size
    for x in range(tamJanela - 1, tamInput[0] - tamJanela + 1):
        for y in range(tamJanela - 1, tamInput[1] - tamJanela + 1):
            scan = []

            for i in range((-1) * floor(tamJanela / 2), ceil(tamJanela / 2)):
                for j in range((-1) * floor(tamJanela / 2), ceil(tamJanela / 2)):
                    scan.append(Input.getpixel((x + i, y + j)))

            Output.putpixel((x, y), round(mean(scan)))
    return Output

Mínima Variância

In [4]:
def minVar(regioes:list) -> list:
    regiaoMinima = regioes[0]
    varRegiaoMinima = variance(regioes[0])
    for mask in regioes:
        if variance(mask) < varRegiaoMinima:
            regiaoMinima = mask
            varRegiaoMinima = variance(mask)
    return regiaoMinima

Função Sal e Pimenta

In [5]:
def salPimenta(imagemOrig:Image, pSal:float, pPimenta:float) -> Image:
    imagemNoised = imagemOrig.copy()
    tam = imagemNoised.size
    for i in range(round(tam[0] * tam[1] * pSal)):
        x = round(random.random() * tam[0]) - 1
        y = round(random.random() * tam[1]) - 1
        imagemNoised.putpixel((x, y), 255)
    for i in range(round(tam[0] * tam[1] * pPimenta)):
        x = round(random.random() * tam[0]) - 1
        y = round(random.random() * tam[1]) - 1
        imagemNoised.putpixel((x, y), 0)
    return imagemNoised

Kuwahara et al. (1976)

In [6]:
def kuwahara(imagemRui:Image, tamJanela:int):
    imagemRec = imagemRui.copy()
    tamImage = imagemRui.size
    for x in range(tamJanela - 1, tamImage[0] - tamJanela + 1):
        for y in range(tamJanela - 1, tamImage[1] - tamJanela + 1):
            masks = [[] for _ in range(qtdMaksKuwahara)]

            for i in range(tamJanela - 1, -1, -1):
                for j in range(tamJanela - 1, -1, -1):
                    masks[0].append(imagemRui.getpixel((x - i, y - j)))
                    masks[1].append(imagemRui.getpixel((x + i, y - j)))
                    masks[2].append(imagemRui.getpixel((x - i, y + i)))
                    masks[3].append(imagemRui.getpixel((x + i, y + j)))
            
            maskMin = minVar(masks)
            media = round(mean(maskMin))
            imagemRec.putpixel((x, y), media)
    return imagemRec

Tomita e Tsuji (1977)

In [10]:
def tomita_tsuji(imagemRui:Image):
    imagemRec = imagemRui.copy()
    tamImage = imagemRui.size
    for x in range(2, tamImage[0] - 2):
        for y in range(2, tamImage[1] - 2):
            masks = [[] for _ in range(qtdMaksTomitaTsuji)]

            for i in range(2, -1, -1):
                for j in range(2, -1, -1):
                    masks[0].append(imagemRui.getpixel((x - j, y - i)))
                    masks[1].append(imagemRui.getpixel((x + j, y - i)))
                    masks[2].append(imagemRui.getpixel((x - j, y + i)))
                    masks[3].append(imagemRui.getpixel((x + j, y + i)))
            
            for i in range(-1, 2):
                for j in range(-1, 2):
                    masks[4].append(imagemRui.getpixel((x + i, y + j)))
            
            maskMin = minVar(masks)
            media = round(mean(maskMin))
            imagemRec.putpixel((x, y), media)
    return imagemRec

Nagao e Matsuyama (1979)

In [14]:
def nagao_matsuyama(imagemRui:Image):
    imagemRec = imagemRui.copy()
    tamImage = imagemRui.size
    for x in range(2, tamImage[0] - 2):
        for y in range(2, tamImage[1] - 2):
            masks = [[] for _ in range(qtdMaksNagaoMatsuyama)]
            
            for i in range(-1, 2):
                for j in range(-1, 2):
                    masks[0].append(imagemRui.getpixel((x + i, y + j)))

            for i in range(-1, 2):
                for j in range(-2, 0):
                    masks[1].append(imagemRui.getpixel((x + i, y + j)))
                    masks[2].append(imagemRui.getpixel((x - j , y + i)))
                    masks[3].append(imagemRui.getpixel((x + i , y - j)))
                    masks[4].append(imagemRui.getpixel((x + j , y + i)))

            for i in range(2, -1, -1):
                for j in range(2, -1, -1):
                    if((j != 0 and i != 2) or (j != 2 and i != 0)):
                        masks[5].append(imagemRui.getpixel((x - j, y - i)))
                        masks[6].append(imagemRui.getpixel((x + j, y - i)))
                        masks[7].append(imagemRui.getpixel((x - j, y + i)))
                        masks[8].append(imagemRui.getpixel((x + j, y + i)))

            maskMin = minVar(masks)
            media = round(mean(maskMin))
            imagemRec.putpixel((x, y), media)
    return imagemRec

Somboonkaew et. al. (1999)

In [ ]:
def somboonkaew(imagemRui:Image):
    imagemRec = imagemRui.copy()
    tamImage = imagemRui.size
    for x in range(2, tamImage[0] - 2):
        for y in range(2, tamImage[1] - 2):
            masks = [[] for _ in range(qtdMaksSomboonkaew)]
            
            

            maskMin = minVar(masks)
            media = round(mean(maskMin))
            imagemRec.putpixel((x, y), media)
    return imagemRec

Debug Kuwahara et al. (1976)

In [9]:
def main():

    # Abertura da imagem
    imagem = Image.open("images/paisagem.jpeg").convert("L")

    # Aplicação do Ruído 5% de Sal e 5% de Pimenta
    imagemRui = salPimenta(imagem, 0.05, 0.05)

    # Salvando a imagem Ruidosa
    imagemRui.save("images/ruidosa.png")

    # Imagem Recuperada com o método de Kuwahara
    imagemRec = kuwahara(imagemRui, 3)

    # Aplicação do Filtro de Suavização
    imagemRec = filtroMedia(imagemRec, 3)

    # Aplicação do Filtro de Realce
    imagemRec = filtroRealce(imagemRec, 3)

    # Salvando a imagem recuperada
    imagemRec.save("images/recuperadaKuwahara.png")


if __name__ == "__main__":
    main()

Debug Tomita e Tsuji (1977)

In [12]:
def main():
    # Abertura da imagem
    imagem = Image.open("images/paisagem.jpeg").convert("L")

    # Aplicação do Ruído 5% de Sal e 5% de Pimenta
    imagemRui = salPimenta(imagem, 0.05, 0.05)

    # Salvando a imagem Ruidosa
    imagemRui.save("images/ruidosa.png")

    # Imagem Recuperada com o método de Tomita e Tsuji
    imagemRec = tomita_tsuji(imagemRui)

    # Aplicação do Filtro de Suavização
    imagemRec = filtroMedia(imagemRec, 3)

    # Aplicação do Filtro de Realce
    imagemRec = filtroRealce(imagemRec, 3)

    # Salvando a imagem recuperada
    imagemRec.save("images/recuperadaTomitaTsuji.png")


if __name__ == "__main__":
    main()

Debug Nagao e Matsuyama (1979)

In [20]:
def main():
    # Abertura da imagem
    imagem = Image.open("images/paisagem.jpeg").convert("L")

    # Aplicação do Ruído 5% de Sal e 5% de Pimenta
    imagemRui = salPimenta(imagem, 0.05, 0.05)

    # Salvando a imagem Ruidosa
    imagemRui.save("images/ruidosa.png")

    # Imagem Recuperada com o método de Nagao e Matsuyama
    imagemRec = nagao_matsuyama(imagemRui)

    # Aplicação do Filtro de Suavização
    #imagemRec = filtroMedia(imagemRec, 3)

    # Aplicação do Filtro de Realce
    imagemRec = filtroRealce(imagemRec, 3)

    # Salvando a imagem recuperada
    imagemRec.save("images/recuperadaNagaoMatsuyama.png")


if __name__ == "__main__":
    main()